<a href="https://colab.research.google.com/github/JelenaBanjac/protein-reconstruction/blob/master/notebooks/colab_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D Protein Pose Recovery

## (0) Setup Colab

In [1]:
running_on_colab = True

In [2]:
from pathlib import Path

if running_on_colab:
    # create directories
    !mkdir --parents data/5j0n
    !mkdir training

    !wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh  #Miniconda3-latest-Linux-x86_64.sh
    !chmod +x Miniconda3-latest-Linux-x86_64.sh
    !bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

    # save the model on the drive
    from google.colab import drive
    drive.mount("/content/drive")
    #!cd 'drive/My Drive'
    !mkdir --parents 'drive/My Drive/ModelsProtein'

    # clone repo
    !rm -r protein-reconstruction
    !git clone https://github.com/JelenaBanjac/protein-reconstruction.git
    !cd protein-reconstruction

    #!conda env create -f protein-reconstruction/environment.yml 

else:
    import sys
    sys.path.append("..") 
    
    DATA_DIR        = "../data"

mkdir: cannot create directory ‘training’: File exists
--2020-08-23 22:37:57--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88867207 (85M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.3-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  84.75M   153MB/s    in 0.6s    

2020-08-23 22:37:58 (153 MB/s) - ‘Miniconda3-py37_4.8.3-Linux-x86_64.sh’ saved [88867207/88867207]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ done

# All requested packages already installed.

installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify 

---

In [3]:
# # activate your conda environment
# %%bash
# source activate protein_reconstruction
# which python


In [4]:
#!conda install -q -y --prefix /usr/local -c conda-forge ipyvolume
##!/usr/local/envs/protein_reconstruction/bin/pip uninstall tensorflow-gpu
##!/usr/local/envs/protein_reconstruction/bin/pip uninstall tensorflow
##!/usr/local/envs/protein_reconstruction/bin/pip install tensorflow-gpu==2.0.0
#!/usr/local/envs/protein_reconstruction/bin/pip3 uninstall tensorflow-gpu
#!/usr/local/envs/protein_reconstruction/bin/pip3 install tensorflow-gpu==2.0.0rc1
#!pip3 uninstall tensorflow-gpu
##!pip uninstall tensorflow-gpu
##!pip uninstall tensorflow
##!pip3 uninstall tensorflow-gpu
##!pip3 uninstall tensorflow

In [12]:
import sys
#sys.path.append("/usr/local/envs/protein_reconstruction/lib/python3.6/site-packages") 
#sys.path.append('/usr/local/lib/python3.7/site-packages')
sys.path.append("protein-reconstruction")

sys.path.remove('/env/python')
#sys.path.remove('/usr/lib/python36.zip')
#sys.path.remove('/usr/lib/python3.6')
#sys.path.remove('/usr/lib/python3.6/lib-dynload')
#sys.path.remove('/usr/local/lib/python3.6/dist-packages') 
#sys.path.remove('/usr/lib/python3/dist-packages') 
#sys.path.remove('/usr/local/lib/python3.6/dist-packages/IPython/extensions')
#sys.path.remove('')

DATA_DIR        = "/content/drive/My Drive/ModelsProtein"

In [6]:
#!pip install tensorflow-graphics

In [7]:
!pip3 install mrcfile

In [13]:
print(sys.path)

['', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', 'protein-reconstruction', '/content', 'protein-reconstruction']


In [14]:
!conda install -c astra-toolbox astra-toolbox

Solving environment: | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 
Found conflicts! Looking for incompatible packages.
This c

In [9]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.python.client import device_lib 
print(tf.__version__)
print(device_lib.list_local_devices())

from cryoem.projections import generate_2D_projections
from cryoem.plots import plot_projection, plot_detector_pixels, plot_angles_count
from cryoem.preprocessing import preprocessing, train_val_test_split
from cryoem.distance_estimation import train_distance_estimation
from cryoem.angle_alignment import training_angle_alignment, update_quaternion
from tensorflow.keras.optimizers import Adagrad, Ftrl

from cryoem.conversions import d_q, euler2quaternion
from cryoem.plots import plot_dP_dQ
import h5py
import ipyvolume as ipv
import numpy as np
print (sys.version)


2.3.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10676781074185096852
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3803621833859401596
physical_device_desc: "device: XLA_CPU device"
]


ModuleNotFoundError: ignored

## (1) Generate Sythetic Data

In [ ]:
PROTEIN         = "5j0n"

PROJECTIONS_NUM = 5000
INPUT_FILE      = f"{PROTEIN}.mrc"
ANGLE_COVERAGE  = [2.0, 1.0, 0.5]
ANGLE_SHIFT     = [0., 0., 0.]
OUTPUT_FILE     = f"{PROTEIN}_ProjectionsAngles_ProjNber{PROJECTIONS_NUM}_AngCoverage{','.join(map(str, ANGLE_COVERAGE))}_AngShift{','.join(map(str, ANGLE_COVERAGE))}.h5"

In [ ]:
# generate sythetic data - 2D projections and their corresponding orientations from a given 3D protein
generate_2D_projections(input_file_path=f"{DATA_DIR}/{INPUT_FILE}", 
                        ProjNber=PROJECTIONS_NUM,
                        AngCoverage=ANGLE_COVERAGE,
                        AngShift=ANGLE_SHIFT,
                        angles_gen_mode="uniform_angles",
                        output_file_name=f"{DATA_DIR}/{OUTPUT_FILE}")

In [ ]:
# read stored data for further processing
data = h5py.File(f"{DATA_DIR}/{OUTPUT_FILE}", 'r')
projections = data['Projections']
angles_true = np.array(data['Angles'])

print(f"{projections.shape[0]} projections of images with dimension {projections.shape[1:]} pixels")
print(f"{angles_true.shape[0]} sets of {angles_true.shape[1]} ground truth rotation angles (Z-Y-Z axes) of corresponding projection images")

In [ ]:
plot_angles_count(angles_true)

In [ ]:
plot_detector_pixels(angles_true)

In [ ]:
ipv.screenshot()

## (2) Preprocessing Data

In [ ]:
NOISY_VAR = 10
TRANSLATION = 5

In [ ]:
plot_projection(projections[0], angles=angles_true[0])

In [ ]:
projections = preprocessing(projections, noise_var=NOISY_VAR, left_limit=-TRANSLATION, peak_limit=0, right_limit=TRANSLATION, channels="gray")

In [ ]:
plot_projection(projections[0], angles=angles_true[0])

## (3) Distance Learning

In [ ]:
DE_LOGS_DIR = "logs/distance_estimation"
DE_TRAINING_DESCRIPTION = "full_pipeline_test"

In [ ]:
train_idx, val_idx, test_idx, train_pairs_num, val_pairs_num = train_val_test_split(PROJECTIONS_NUM, test_size=0.33, val_size=0.25, train_percent=0.01, val_percent=0.01)

In [ ]:
model, history = train_distance_estimation(X=projections, y=angles_true, 
                                           train_idx=train_idx, 
                                           val_idx=val_idx,
                                           epochs=2,  #500
                                           batch_size=256,  #512 latest 
                                           learning_rate=1e-3, 
                                           limit_style="random", 
                                           path_logs_training=DE_LOGS_DIR, 
                                           training_description=DE_DESCRIPTION,
                                           training_steps=train_pairs_num,
                                           validation_steps=val_pairs_num, 
                                           plot=True,
                                           gpus=None)

In [ ]:
# read stored model
model_filename = f"{METHODS_LOGS_DIR}/training/{TRAINING_DESCRIPTION}.h5"

# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model(model_filename)

# Show the model architecture
model.summary()

In [ ]:
# learned distance function
def d_p(p1, p2):
    p1 = tf.cast(p1, dtype=tf.float32)
    p2 = tf.cast(p2, dtype=tf.float32)
    return model.predict([p1, p2], batch_size=512)

In [ ]:
# dP/dQ ratio plot
batch_size = 512
idx1 = list(np.random.choice(range(5000), size=batch_size))
idx2 = list(np.random.choice(range(5000), size=batch_size))

q1_true = euler2quaternion([angles_true[i] for i in idx1])
q2_true = euler2quaternion([angles_true[i] for i in idx2])

p1 = [projections[i] for i in idx1]
p2 = [projections[i] for i in idx2]

dP_values = d_p(p1, p2).T[0]
dQ_values = d_q(q1_true, q2_true).numpy()

plot_dP_dQ(dP_values, dQ_values);

## (4) Angle Recovery

In [ ]:
AR_LOGS_DIR = "logs/angle_recovery"
AR_TRAINING_DESCRIPTION = "full_pipeline_test"

In [ ]:
q_predicted, losses, q_all = train_angle_recovery(steps=50000, 
                                                 batch_size=256, 
                                                 in_data=projections, 
                                                 distance_fn=d_p, 
                                                 learning_rate=0.5, 
                                                 limit_distance=np.pi,
                                                 file_name=f"{AR_LOGS_DIR}/{AR_TRAINING_DESCRIPTION}")

In [ ]:
angles_predicted = quaternion2euler(q_predicted)

## (5) Angle Alignment

In [ ]:
m, a_R, losses, collect_data, trajectory = training_angle_alignment(num_runs=3, 
                                                                    steps=300, 
                                                                    batch_size=256, 
                                                                    optimizer=Ftrl(learning_rate=2., learning_rate_power=-2.), 
                                                                    angles_true=angles_true, 
                                                                    angles_predicted=angles_predicted)

In [ ]:
m = m
trajectory_first = trajectory[0]
loss_first = losses[0]
trajectory_last = trajectory[-1]
loss_last = losses[-1]

m, trajectory_first, loss_first, trajectory_last, loss_last

In [ ]:
q_predicted_rotated =  update_quaternion(m, a_R, q_predicted)
angles_predicted_rotated = quaternion2euler(q_predicted_rotated)

## (6) Reconstruction

In [ ]:
REC_LOGS_DIR = "logs/reconstruction"
REC_DESCRIPTION = "full_pipeline_test.mrc"

In [ ]:
reconstruct(projections, angles_predicted, mrc_filename=f"{REC_LOGS_DIR}/{REC_DESCRIPTION}")